In [1]:
import json
import time
import requests
import pandas as pd
import re
import string

In [2]:
# 과학적 표기법 사용안함
pd.options.display.float_format = '{:.2f}'.format

In [102]:
# API로 데이터 불러오기
start_time = time.time()
headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36'}
limit = 0
api_url = f'https://api.nasdaq.com/api/screener/stocks?tableonly=true&limit={limit}'
api = requests.get(api_url, headers=headers)
print(f"{time.time()-start_time} s")

3.4493987560272217 s


In [103]:
start_time = time.time()
# deserialization
data = json.loads(api.text)
print(f"{time.time()-start_time} s")

0.029892921447753906 s


In [5]:
# 상장 기업 수 체크
start_time = time.time()
num_of_listed_comp = data['data']['totalrecords']
print(f"{time.time()-start_time} s")

0.0002639293670654297 s


In [6]:
start_time = time.time()
num_of_listed_comp = data.get('data').get('totalrecords') # which is faster?
print(f"{time.time()-start_time} s")

7.796287536621094e-05 s


In [7]:
print(num_of_listed_comp)

7783


In [8]:
# 상장 기업 리스트
d = data['data']['table']['rows']

In [81]:
start_time = time.time()
df = pd.DataFrame(d)
print(f"{time.time()-start_time} s")

0.0273439884185791 s


In [82]:
comp_list = df.iloc[:,:2]
comp_list.head()

,symbol,name
0,AAPL,Apple Inc. Common Stock
1,MSFT,Microsoft Corporation Common Stock
2,AMZN,"Amazon.com, Inc. Common Stock"
3,GOOG,Alphabet Inc. Class C Capital Stock
4,GOOGL,Alphabet Inc. Class A Common Stock


In [83]:
"""
/s : 공백 매칭
| : or
\(([\w].*)\)* : 괄호 안에 있는 문자열 포함, 뒤에 있는 모든 문자열
(\sClass [\w].*) : Class라는 문자열 포함, 뒤에 있는 모든 문자열
(Limited)?(\s[\w]*)(\sStock|\sShares).* : Limited가 있거나 Stock이나 Shares 앞에오는 단어 하나 포함, 뒤에 있는 모든 문자열 
(American (Depositary|Depository)).* : American Depositary 또는 American Depository 포함, 뒤에 있는 모든 문자열
"""

# regex = re.compile('''
#     \s\(([\w].*)\)*|
#     (\sClass [\w].*)|
#     (\sLimited|\sNew)?(\s[\w]*)(\sStock|\sShares).*|
#     (\sAmerican\s(Depositary|Depository)).*
#     ''')

# pattern = r'\,|\s\(([\w].*)\)*|\sClass\s[\w].*|(\bCommon\b).*|(\sNew)?(\s[\w]*)?(\sStock|\sShare|\sADR).*|\sAmerican\s(Depositary|Depository).*'
pattern = r'\,|\s\(([\w].*)\)*|(?<=inc.).*|(\bcommon\b|\bclass\b).*|(\sNew)?(\s[\w]*)?(\sstock|\sshare|(\sADR|\sADS)).*|([\w]*)(\sdepository|\sdepositary).*'
regex = re.compile(pattern, flags=re.IGNORECASE)

In [84]:
start_time = time.time()
for i in comp_list.values:
    try:
        i[1] = regex.sub('', i[1])
        i[1] = ' '.join(i[1].split())
    except Exception as e:
        print(e)
        break;
print(f"{time.time()-start_time} s")

0.11974287033081055 s


In [85]:
for i in comp_list.values:
    print(i)

['AAPL' 'Apple Inc.']
['MSFT' 'Microsoft Corporation']
['AMZN' 'Amazon.com Inc.']
['GOOG' 'Alphabet Inc.']
['GOOGL' 'Alphabet Inc.']
['FB' 'Facebook Inc.']
['TSM' 'Taiwan Semiconductor Manufacturing Company Ltd.']
['BABA' 'Alibaba Group Holding Limited']
['TSLA' 'Tesla Inc.']
['JPM' 'JP Morgan Chase & Co.']
['V' 'Visa Inc.']
['JNJ' 'Johnson & Johnson']
['NVDA' 'NVIDIA Corporation']
['WMT' 'Walmart Inc.']
['UNH' 'UnitedHealth Group Inco']
['BAC' 'Bank of America Corporation']
['MA' 'Mastercard Inco']
['HD' 'Home Depot Inc.']
['PG' 'Procter & Gamble Company']
['DIS' 'Walt Disney Company']
['PYPL' 'PayPal Holdings Inc.']
['ASML' 'ASML Holding N.V. New York']
['CMCSA' 'Comcast Corporation']
['XOM' 'Exxon Mobil Corporation']
['TM' 'Toyota Motor Corporation']
['KO' 'Coca-Cola Company']
['ADBE' 'Adobe Inc.']
['VZ' 'Verizon Communications Inc.']
['ORCL' 'Oracle Corporation']
['INTC' 'Intel Corporation']
['CSCO' 'Cisco Systems Inc.']
['NFLX' 'Netflix Inc.']
['CRM' 'Salesforce.com Inc']
['PFE' '

['NVG' 'Nuveen AMT-Free Municipal Credit Inco']
['ARNA' 'Arena Pharmaceuticals Inc.']
['ALKS' 'Alkermes plc']
['BCO' 'Brinks Company']
['TEX' 'Terex Corporation']
['LXP' 'Lexington Realty Trust']
['SXT' 'Sensient Technologies Corporation']
['CWH' 'Camping World Holdings Inc.']
['FHB' 'First Hawaiian Inc.']
['WTM' 'White Mountains Insurance Group Ltd.']
['SWIM' 'Latham Group Inc.']
['SFBS' 'ServisFirst Bancshares Inc.']
['FSKR' 'FS KKR Capital Corp. II']
['URBN' 'Urban Outfitters Inc.']
['SPT' 'Sprout Social Inc']
['NSIT' 'Insight Enterprises Inc.']
['KNSL' 'Kinsale Capital Group Inc.']
['SAFM' 'Sanderson Farms Inc.']
['CLOV' 'Clover Health Investments Corp.']
['ISBC' 'Investors Bancorp Inc.']
['RRC' 'Range Resources Corporation']
['MRCY' 'Mercury Systems Inc']
['KTB' 'Kontoor Brands Inc.']
['ATKR' 'Atkore Inc.']
['AGI' 'Alamos Gold Inc.']
['MAC' 'Macerich Company']
['ACAD' 'ACADIA Pharmaceuticals Inc.']
['TRTN' 'Triton International Limited']
['STEM' 'Stem Inc.']
['APLE' 'Apple Hospita

['CGBD' 'TCG BDC Inc.']
['PCN' 'Pimco Corporate & Inco']
['ETH' 'Ethan Allen Interiors Inc.']
['ADAP' 'Adaptimmune Therapeutics plc']
['OXLC' 'Oxford Lane Capital Corp.']
['CAI' 'CAI International Inc.']
['ONEW' 'OneWater Marine Inc.']
['VOR' 'Vor Biopharma Inc.']
['SP' 'SP Plus Corporation']
['REV' 'Revlon Inc.']
['TWCT' 'TWC Tech Holdings II Corp.']
['PRAX' 'Praxis Precision Medicines Inc.']
['ETAC' 'E.Merge Technology Acquisition Corp.']
['BOAC' 'Bluescape Opportunities Acquisition Corp.']
['VTA' 'Invesco Credit Opportunities Fund']
['BOWX' 'BowX Acquisition Corp.']
['CAPL' 'CrossAmerica Partners LP']
['SCSC' 'ScanSource Inc.']
['BBAR' 'Banco BBVA Argentina S.A.']
['FMBH' 'First Mid Bancshares Inc.']
['GCI' 'Gannett Co. Inc.']
['FDMT' '4D Molecular Therapeutics Inc.']
['CO' 'Global Cord Blood Corporation']
['AGC' 'Altimeter Growth Corp.']
['TRMD' 'TORM plc']
['AVAN' 'Avanti Acquisition Corp.']
['FFIC' 'Flushing Financial Corporation']
['MITK' 'Mitek Systems Inc.']
['BZH' 'Beazer Hom

['ATLO' 'Ames National Corporation']
['IRS' 'IRSA Inversiones Y Representaciones S.A.']
['MFIN' 'Medallion Financial Corp.']
['FTFT' 'Future FinTech Group Inc.']
['AIH' 'Aesthetic Medical International Holdings Group Ltd.']
['FIF' 'First Trust Energy Infrastructure Fund']
['SCPE' 'SC Health Corporation']
['USAS' 'Americas Gold and Silver Corporation']
['VBF' 'Invesco Bond Fund']
['IVH' 'Ivy High Inco']
['NGC' 'Northern Genesis Acquisition Corp. III']
['VEL' 'Velocity Financial Inc.']
['LBPS' '4D pharma plc']
['CMLS' 'Cumulus Media Inc.']
['LCNB' 'LCNB Corporation']
['CPLP' 'Capital Product Partners L.P.']
['MFNC' 'Mackinac Financial Corporation']
['BWFG' 'Bankwell Financial Group Inc.']
['NETI' 'Eneti Inc.']
['CODX' 'Co-Diagnostics Inc.']
['CNF' 'CNFinance Holdings Limited']
['GWGH' 'GWG Holdings Inc']
['MYFW' 'First Western Financial Inc.']
['HURC' 'Hurco Companies Inc.']
['ATNF' '180 Life Sciences Corp.']
['EDAP' 'EDAP TMS S.A.']
['TSQ' 'Townsquare Media Inc.']
['CASI' 'CASI Pharmace

['ADOCW' 'Edoc Acquisition Corp. Warrant']
['ADVWW' 'Advantage Solutions Inc.']
['AEACU' 'Authentic Equity Acquisition Corp. Unit']
['AEACW' 'Authentic Equity Acquisition Corp. Warrant']
['AEB' 'AEGON N.V. Perp. Cap. Secs. Floating Rate']
['AEFC' 'Aegon Funding Company LLC 5.10% Subordinated Notes due 2049']
['AEL^A' 'American Equity Investment Life Holding']
['AEL^B' 'American Equity Investment Life Holding']
['AEPPL' 'American Electric Power Company Inc.']
['AEPPZ' 'American Electric Power Company Inc.']
['AESC' 'The AES Corporation Corporate Units']
['AESR' 'Anfield U.S. Equity Sector Rotation ETF']
['AFAQU' 'AF Acquisition Corp. Units']
['AFAQW' 'AF Acquisition Corp. Warrants']
['AFGB' 'American Financial Group Inc.']
['AFGD' 'American Financial Group Inc.']
['AFGE' 'American Financial Group Inc.']
['AFINO' 'American Finance Trust Inc.']
['AFINP' 'American Finance Trust Inc.']
['AGBAR' 'AGBA Acquisition Limited Right']
['AGBAW' 'AGBA Acquisition Limited Warrant']
['AGCUU' 'Altimete

['TMPMU' 'Turmeric Acquisition Corp. Unit']
['TMPMW' 'Turmeric Acquisition Corp. Warrant']
['TMTSU' 'Spartacus Acquisition Corporation Unit']
['TMTSW' 'Spartacus Acquisition Corporation Warrant']
['TNP^D'
 'Tsakos Energy Navigation Ltd 8.75% Series D Cumulative Redeemable Perpetual']
['TNP^E'
 'Tsakos Energy Navigation Ltd Series E Fixed-to-Floating Rate Cumulative Redeemable Perpetual']
['TNP^F'
 'Tsakos Energy Navigation Ltd Series F Fixed-to-Floating Rate Cumulative Redeemable Perpetual']
['TRITW' 'Triterras Inc.']
['TRKAW' 'Troika Media Group Inc.']
['TRTN^A'
 'Triton International Limited 8.50% Series A Cumulative Redeemable Perpetual']
['TRTN^B'
 'Triton International Limited 8.00% Series B Cumulative Redeemable Perpetual']
['TRTN^C'
 'Triton International Limited 7.375% Series C Cumulative Redeemable Perpetual']
['TRTN^D'
 'Triton International Limited 6.875% Series D Cumulative Redeemable Perpetual']
['TSCBP' 'TriState Capital Holdings Inc.']
['TSIAU' 'TS Innovation Acquisition

In [91]:
for i in comp_list['symbol']:
    print(i)

AAPL
MSFT
AMZN
GOOG
GOOGL
FB
TSM
BABA
TSLA
JPM
V
JNJ
NVDA
WMT
UNH
BAC
MA
HD
PG
DIS
PYPL
ASML
CMCSA
XOM
TM
KO
ADBE
VZ
ORCL
INTC
CSCO
NFLX
CRM
PFE
NKE
T
CVX
PEP
NVS
ABBV
WFC
AVGO
BHP
MRK
LLY
ABT
UPS
NVO
EDU
ACN
TMO
TMUS
TXN
PDD
MCD
DHR
MS
COST
MDT
C
SAP
HON
UL
BBL
LIN
SHOP
BUD
PM
QCOM
BA
UNP
RY
AZN
RIO
BMY
NEE
CSAN
SCHW
CHTR
HDB
LOW
SE
BLK
AMGN
RTX
CAT
SBUX
TD
AXP
HSBC
SNY
IBM
GS
TOT
AMAT
GE
INTU
SONY
JD
AMT
MMM
VALE
DEO
CVS
TGT
DE
EL
LMT
SQ
AMD
SNAP
ISRG
GSK
ZM
ANTM
UBER
BKNG
SYK
MU
BP
FIS
LRCX
NOW
MO
SPGI
USB
BTI
PLD
MDLZ
ABNB
CI
GM
ADP
CCI
PNC
GILD
ZTS
BNS
TFC
INFY
FDX
TJX
PTR
COP
CNI
BAM
ENB
DUK
DELL
NTES
CME
CB
MRNA
FISV
CSX
WBK
SHW
MUFG
COF
EQNR
ATVI
ITW
SAN
SNOW
PBR
CPNG
MMC
HCA
ABB
CL
NSC
BDX
BMO
BIDU
SO
MELI
APD
EQIX
NIO
SNP
VMW
BX
BSBR
FCX
MCO
ICE
STLA
IBN
D
ADSK
ECL
ADI
BEKE
ILMN
ITUB
ABEV
BSX
WM
NOC
LFC
F
NEM
PGR
TEAM
EW
EMR
BBD
ETN
GPN
MET
UBS
RBLX
TWLO
AON
NXPI
WDAY
PUK
HMC
SCCO
HUM
ING
CP
VRTX
KHC
GD
CM
REGN
EPD
RACE
TAK
KDP
BNTX
DOW
VOD
RELX
AMX
TRP
PHG
PSA
SMFG
MNST
COIN

HESM
CYD
PEBO
SFT
NXJ
CGEN
LBC
DCO
HIFS
PDFS
SPLP
CNBKA
CMTL
OPY
TREB
VERI
VGAC
KDMN
GHY
ACRE
CURI
FVIV
PGC
SXC
CMO
GHVI
DIAX
MMD
CHW
DBVT
DTIL
LEAP
MSBI
RIGL
WIFI
LOCO
FOA
ISEE
NBB
IKNA
VSEC
AUD
KNOP
ACIC
SOS
MVF
AWH
CPLG
BOLT
LANDM
CFIV
HIX
MRNS
REPX
CCV
DX
VYGG
WTI
ALBO
ACQR
SIFY
SPPI
TDW
THFF
NMM
BLW
UBP
MIT
EOLS
BGB
CATC
NBTX
TPGY
DXPE
FSNB
BLX
HPS
NSSC
MGTA
TWI
CCRN
EBON
VEC
CRAI
IPOD
MUI
GTS
IMXI
ADAG
AGCB
LPG
GSL
IFN
MTL
JIH
REX
MUA
ASPN
NAKD
THW
RMGC
LOTZ
LMACA
THR
EGLE
MMU
AMRK
BIOX
ESPR
CDXC
ZEV
BME
JPI
SWIR
CTT
TRUE
QTT
BCOV
ICPT
STXS
QD
CPRX
NDLS
ALRS
NVGS
GTPB
DPG
SNR
FSP
DMRC
GILT
VIOT
ACBI
USX
XENT
PPR
NPCE
DHIL
EFT
PACE
TTCF
NAT
TSPQ
CSWC
OTRK
AWP
ASXC
PLYM
DFP
REVH
CEIX
SEAH
BEST
HMLP
KVSC
AROW
FRPH
BBSI
EVN
MPX
SCOA
IGD
PROS
ATER
VRS
NQP
CSTE
SMTS
OPRT
LEO
PKO
MBI
INSW
BRY
GMII
VAQC
STK
GLOG
HAAC
MFL
CLVS
HPI
BFC
SOL
JOAN
DLTH
DMLP
AVD
FCBC
KE
EBF
MYN
LAWS
ARLO
JFR
RMNI
PRPC
OG
OLP
ATRO
VKI
PERI
MCB
ZYXI
GMTX
TRIT
NRGX
VECT
APEI
NLS
RMT
PRTH
RDIB
CDZI
HCKT
ISD
UONE
A

SND
RYB
PLX
APVO
JLS
ISR
JMP
COCP
SFBC
GASS
ALRN
IAE
NWHM
AMRB
RELL
VISL
LUB
VYNT
MSAC
VCIF
ORGS
OVBC
LFVN
EDUC
AEZS
CCRC
CMT
CYCN
MKD
CPSS
PFX
LPCN
NREF
RAIL
EXD
SCPS
PBIP
BKSC
LXEH
BIOL
CHAQ
WTER
FSLF
PPBT
UBOH
CRVS
REPH
NMI
SMHI
FVAM
NXC
RZLT
CIZN
APT
PGZ
DTEA
SFUN
GFED
UGRO
MBRX
CWBC
ARAV
CXDO
ARMP
ADMP
DTF
MTCR
CLSN
PETZ
ONTX
APM
SMM
XAIR
MMLP
FAM
MIE
MCHX
CLDB
CCNC
DYAI
OSS
LIVK
LYRA
OCG
LSAQ
HGBL
CGA
GLG
MAGS
EPSN
PHUN
ARC
PNF
SLNO
ETTX
WPG
MKTY
CSCW
DLNG
PCSA
APRE
RSF
KRKR
REED
ISDR
GTT
LMB
HMNF
OBLG
GLV
CDTX
AIKI
CEV
AXR
AVGR
XELA
AACG
SMID
CODA
EXN
CSU
ISSC
LFT
RIVE
SSBI
AVCO
LPTX
INUV
USAP
TCON
AESE
CORR
UAMY
IFBD
ITAC
TRKA
QRHC
CPSH
ESEA
AAME
CGRN
NMS
APRN
LNSR
PCF
MDWD
PHX
VLT
FRD
BHSE
CCLP
MITO
CXH
BTN
CLBS
GTEC
FXNC
GROW
PAIC
CAPR
DDF
UBCP
FEIM
ZGYH
PTE
SYNL
BWEN
TPZ
DGLY
HWCC
GRNQ
PNRG
WFCF
FNHC
CLRB
LWAY
INTG
RWLK
RMTI
EDI
CLPS
ELMD
NDRA
BBIG
AFBI
VJET
LAIX
BAOS
ELYS
PESI
HNRG
LUMO
FMO
IHTA
ECOR
LACQ
MFD
LSBK
ARPO
ZSAN
NAII
MACK
KINS
QUMU
RCON
RGT
ISUN
GRAY
USAU
RCAT
O